In [14]:
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as T
import os
from PIL import Image
from torch.utils.data import DataLoader

### DATASET

In [13]:
class CarlaDataset(Dataset):
    def __init__(self, csv_file, img_dir):
        self.train_data = pd.read_csv(csv_file)
        self.dir_img = img_dir

        self.transform = T.Compose([
            T.Resize((66, 200)),
            T.ToTensor(),
            T.Normalize(mean=[0.5, 0.5, 0.5],
                        std=[0.5, 0.5, 0.5])
        ])

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):

        img_path = os.path.join(self.dir_img, self.train_data.iloc[idx, 0])
        img = img = Image.open(img_path)
        img = self.transform(img)

        steering = torch.FloatTensor(self.train_data.iloc[idx, 1])
        return img, steering

### PILOT NET STRUCTURE

In [10]:
class PilotNet(nn.Module):
    def __init__(self):
        super(PilotNet, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ReLU(),

            nn.Conv2d(24, 36, 5, stride=2),
            nn.ReLU(),

            nn.Conv2d(36, 48, 5, stride=2),
            nn.ReLU(),

            nn.Conv2d(48, 64, 3),
            nn.ReLU(),

            nn.Conv2d(64, 64, 3),
            nn.ReLU(),
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 1 * 18, 100),
            nn.ReLU(),

            nn.Linear(100, 50),
            nn.ReLU(),

            nn.Linear(50, 10),
            nn.ReLU(),

            nn.Linear(10, 1)
        )
        
    def forward(self, x):
        x = self.conv(x)
        steering = self.fc(x)

        return steering

### TRAINING

In [ ]:
dataset = CarlaDataset(
    csv_file="_dataset/labels.csv",
    img_dir="_dataset/images"
)

loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [11]:
model = PilotNet()

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Move  model to device
model = model.to(device)
print(model)

cpu
PilotNet(
  (conv): Sequential(
    (0): Conv2d(3, 24, kernel_size=(5, 5), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(24, 36, kernel_size=(5, 5), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(36, 48, kernel_size=(5, 5), stride=(2, 2))
    (5): ReLU()
    (6): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU()
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1152, out_features=100, bias=True)
    (2): ReLU()
    (3): Linear(in_features=100, out_features=50, bias=True)
    (4): ReLU()
    (5): Linear(in_features=50, out_features=10, bias=True)
    (6): ReLU()
    (7): Linear(in_features=10, out_features=1, bias=True)
  )
)


/home/jrguezg/pytorch/lib/python3.10/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 12020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
# Define loss function
criterion = nn.MSELoss()

# Define optimizer
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
epochs = 10

for epoch in range(epochs):
    model.train()
    loss_epoch = 0.0

    for images, steering in loader:

        batch_size = steering.size

        images, steering = images.to(device), steering.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        
        loss = criterion(outputs, steering)
        
        loss.backward()
 
        optimizer.step()

        loss_epoch += loss.item() * batch_size
        
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss_epoch/len(loader):.6f}")

torch.save(model.state_dict(), "pilotnet_steering.pth")